In [4]:
import pandas as pd
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
import os
from sklearn.metrics import classification_report
import boto3

In [5]:
df = pd.read_csv("water.csv")

In [12]:
(df.isna()==True).any()

ph                  True
Hardness           False
Solids             False
Chloramines        False
Sulfate             True
Conductivity       False
Organic_carbon     False
Trihalomethanes     True
Turbidity          False
Potability         False
dtype: bool

In [13]:
df = df.fillna(df.mean())

In [16]:
Y = df['Potability']
X = df.drop(columns=['Potability'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=33)

In [19]:

# SVM Classifier  
def svm_classifier(train_x, train_y):
    model = SVC(kernel='rbf', probability=True)  
    model.fit(train_x, train_y)  
    return model

In [20]:
# training...
model = svm_classifier(X_train,y_train)
y_pred = model.predict(X_test)

# evaluating...
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      1.00      0.77       612
           1       0.33      0.00      0.01       371

    accuracy                           0.62       983
   macro avg       0.48      0.50      0.39       983
weighted avg       0.51      0.62      0.48       983



In [22]:
# concating...
X_test = X_test.reset_index(drop=True)
y_predt = DataFrame(y_pred)
y_predt.columns = ['predict']
result = pd.concat([X_test, y_predt], axis=1)
print("save the predict result!")
result.to_csv('result.csv', index=0)

save the predict result!


In [25]:
# uploading...
aws_bucket_name = 'liuyixin'
s3 = boto3.client('s3',
                  aws_access_key_id='E441571B4C6B777EB1F8',
                  aws_secret_access_key='WzY1QTg4N0JBOTBGQUJFOUNDQTZERTlFMDRBMkI1',
                  endpoint_url='http://scut.depts.bingosoft.net:29997')
s3.upload_file('result.csv', 'liuyixin', 'result.csv')
print('upload succeed!')

upload succeed!
